In [2]:
# %pip install python-dateutil
# %pip install faker

In [16]:
from datetime import datetime
import pandas as pd
from faker import Faker
import random 

fake = Faker(random_state=24)
fecha_minima = datetime.strptime('2006-11-15', '%Y-%m-%d')
fecha_maxima = datetime.strptime('2024-04-01', '%Y-%m-%d')

amount_predefined = [10, 15, 35, 50]
amount_free = [1, 5, 20, 70, 90, 100] 
amount_choices = amount_predefined + amount_free

data2 = []
partner_donation_dates = {}  # Diccionario para realizar seguimiento de las fechas de donación de las empresas asociadas

for _ in range(1000):
    name = fake.name()
    try:
        first_name, last_name = name.split()
    except ValueError:
        first_name = name
        last_name = ""
    email = f"{first_name.lower()}.{last_name.lower()}@gmail.com"
    date = fake.date_between_dates(date_start=fecha_minima, date_end=fecha_maxima)
    is_partner = fake.random_element([0, 1])
    donation_frecuency = fake.random_element([0, 1])

    if is_partner == 1:
        suscription_status = 1
        company = fake.random_element(['Pepsi', 'Amazon', 'Jonhsons', 'Nasscom', 'Vodafone', 'Baxter', 'The Oberoi', 'CBN', 'Hyatt', 'Westin', 'YCI', 'CaSa', 'Cromax', 'Nike', 'Cosco', 'Indigo'])
        role_company = fake.random_element(['CEO',' CTO', 'Employee'])
        donation_type = fake.random_element([1, 2, 3, 4])
        if donation_type == 1:
            amount = fake.random_int(min=200, max=2000)
            method_pay = fake.random_element([1,2,3])
        else:
            amount = 0
            if company == 'Amazon':
                donation_type = 2
        
    else:
        suscription_status = 1 if donation_frecuency == 1 else 0
        company = 'None'
        role_company = 'None'
        donation_type = 1
        amount = fake.random_element(amount_choices)        
        method_pay = fake.random_element([1,2,3])
        donation_date = fake.date_between_dates(date_start=fecha_minima, date_end=fecha_maxima)
    
    data2.append([name, email, date, is_partner, company, role_company, donation_frecuency, suscription_status, donation_type, amount, method_pay, donation_date])


df2 = pd.DataFrame(data2, columns=['name', 'email', 'date', 'is_partner',  'company', 'role_company',  'donation_frecuency', 'suscription_status', 'donation_type', 'amount', 'method_pay','donation_date'])
df2

,name,email,date,is_partner,company,role_company,donation_frecuency,suscription_status,donation_type,amount,method_pay,donation_date
0,Paul Pitts,paul.pitts@gmail.com,2012-07-16,0,None,None,0,0,1,100,1,2016-01-25
1,Claire Peters,claire.peters@gmail.com,2020-06-14,0,None,None,1,1,1,100,2,2010-02-18
2,George Beasley,george.beasley@gmail.com,2015-06-16,0,None,None,0,0,1,10,3,2016-04-27
3,Carolyn Torres,carolyn.torres@gmail.com,2012-10-17,1,Westin,CEO,1,1,1,1776,1,2016-04-27
4,Kimberly Carrillo,kimberly.carrillo@gmail.com,2020-10-08,1,Cosco,CTO,0,1,4,0,1,2016-04-27
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Allen Cox,allen.cox@gmail.com,2013-08-31,1,YCI,CEO,1,1,3,0,2,2021-07-06
996,Ricardo Reynolds,ricardo.reynolds@gmail.com,2020-02-05,1,Hyatt,Employee,0,1,3,0,2,2021-07-06
997,Leonard Kim,leonard.kim@gmail.com,2010-03-16,1,YCI,CEO,1,1,3,0,2,2021-07-06
998,Crystal Myers,crystal.myers@gmail.com,2013-02-19,1,YCI,CTO,0,1,3,0,2,2021-07-06


In [29]:
df2[df2['company'] == 'Pepsi']

,name,email,date,is_partner,company,role_company,donation_frecuency,suscription_status,donation_type,amount,method_pay,donation_date
48,Michael Smith,michael.smith@gmail.com,2011-04-01,1,Pepsi,Employee,1,1,1,1575,3,2021-12-30
53,Robin Carpenter,robin.carpenter@gmail.com,2011-04-01,1,Pepsi,CEO,1,1,1,1733,2,2015-04-04
55,Stephen Woods,stephen.woods@gmail.com,2011-04-01,1,Pepsi,CTO,0,1,2,0,2,2008-12-07
103,Nicole Ward,nicole.ward@gmail.com,2011-04-01,1,Pepsi,CTO,0,1,4,0,3,2016-11-08
111,Nicholas Webb,nicholas.webb@gmail.com,2011-04-01,1,Pepsi,CEO,1,1,1,1024,1,2007-07-18
113,James Mccoy,james.mccoy@gmail.com,2011-04-01,1,Pepsi,CEO,1,1,2,0,1,2020-07-24
165,Holly Lambert,holly.lambert@gmail.com,2011-04-01,1,Pepsi,CEO,1,1,4,0,2,2011-08-10
204,Hannah Taylor,hannah.taylor@gmail.com,2011-04-01,1,Pepsi,Employee,0,1,1,613,2,2014-04-27
298,Gregory Herrera,gregory.herrera@gmail.com,2011-04-01,1,Pepsi,Employee,1,1,4,0,3,2013-01-19
313,Katie Mendoza,katie.mendoza@gmail.com,2011-04-01,1,Pepsi,CEO,1,1,1,629,2,2008-08-03


In [22]:
# Filtrar solo las filas donde is_partner es igual a 1
partner_df = df2[df2['is_partner'] == 1]

# Obtener una lista de empresas únicas
unique_companies = partner_df['company'].unique()

# Iterar sobre las empresas únicas
for company in unique_companies:
    # Obtener la primera fecha de donación para esta empresa
    first_donation_date = partner_df.loc[partner_df['company'] == company, 'date'].iloc[0]
    
    # Actualizar la columna de fecha de donación para todas las filas de esta empresa
    df2.loc[(df2['is_partner'] == 1) & (df2['company'] == company), 'date'] = first_donation_date

# Ahora df2 tiene la fecha de donación de la primera fila de cada empresa para todas las filas de esa empresa


In [25]:
from dateutil.relativedelta import relativedelta
import random 
# Primero, seleccionamos los registros que cumplen las condiciones
filtered_df = df2[(df2['is_partner'] == 0) & (df2['donation_frecuency'] == 1)]

# Luego, creamos una lista para almacenar los registros duplicados con modificaciones
duplicated_records = []

# Iteramos sobre los registros seleccionados y creamos duplicados con modificaciones
for _, row in filtered_df.iterrows():
    # Generamos un valor aleatorio para la cantidad de duplicados
    num_duplicates = random.randint(2, 24)  # Por ejemplo, generamos un número aleatorio entre 1 y 10
    for i in range(num_duplicates):
        duplicated_row = row.copy()
        duplicated_row['donation_date'] += relativedelta(months=i)
        duplicated_records.append(duplicated_row)
            

# El primer ciclo for itera sobre cada fila (registro) en el DataFrame filtered_df, 
# que contiene solo las filas que cumplen con las condiciones especificadas ((df2['is_partner'] == 0) & (df2['donation_frecuency'] == 1)).

# El segundo ciclo for itera sobre los valores en la lista num_duplicates_list. Esta lista contiene la cantidad de duplicados que se deben crear 
# para cada fila seleccionada en el primer ciclo for.

# El tercer ciclo for, dentro del segundo ciclo, itera num_duplicates veces. num_duplicates es el valor actual de la lista num_duplicates_list 
# en cada iteración del segundo ciclo. Este ciclo se utiliza para crear múltiples duplicados de la misma fila.

# Convertimos la lista de registros duplicados en un DataFrame
duplicated_df = pd.DataFrame(duplicated_records)

# Concatenamos el DataFrame duplicado con el DataFrame original
result_df = pd.concat([df2, duplicated_df], ignore_index=True)
result_df.head()

,name,email,date,is_partner,company,role_company,donation_frecuency,suscription_status,donation_type,amount,method_pay,donation_date
0,Paul Pitts,paul.pitts@gmail.com,2012-07-16,0,None,None,0,0,1,100,1,2016-01-25
1,Claire Peters,claire.peters@gmail.com,2020-06-14,0,None,None,1,1,1,100,2,2010-02-18
2,George Beasley,george.beasley@gmail.com,2015-06-16,0,None,None,0,0,1,10,3,2016-04-27
3,Carolyn Torres,carolyn.torres@gmail.com,2012-10-17,1,Westin,CEO,1,1,1,1776,1,2016-04-27
4,Kimberly Carrillo,kimberly.carrillo@gmail.com,2020-10-08,1,Cosco,CTO,0,1,4,0,1,2016-04-27


In [26]:
from dateutil.relativedelta import relativedelta
import random 

# Primero, seleccionamos los registros que cumplen las condiciones
filtered_df = result_df[result_df['is_partner'] == 1]

# Luego, creamos una lista para almacenar los registros duplicados con modificaciones
duplicated_records = []

# Iteramos sobre los registros seleccionados y creamos duplicados con modificaciones
for _, row in filtered_df.iterrows():
    # Duplicamos el registro 10 veces cada 6 meses
    num_duplicates = random.randint(8, 20)
    for i in range(num_duplicates):
        # Calculamos la nueva fecha de donación sumando 6 meses al registro original
        new_donation_date = row['donation_date'] + relativedelta(months=6*i)
        
        # Creamos una copia del registro original y actualizamos la fecha de donación
        duplicated_row = row.copy()
        duplicated_row['donation_date'] = new_donation_date
        
        # Agregamos el registro duplicado a la lista
        duplicated_records.append(duplicated_row)
            
# Convertimos la lista de registros duplicados en un DataFrame
duplicated_df = pd.DataFrame(duplicated_records)

# Concatenamos el DataFrame duplicado con el DataFrame original
result_df2 = pd.concat([result_df, duplicated_df], ignore_index=True)
result_df2.head()


,name,email,date,is_partner,company,role_company,donation_frecuency,suscription_status,donation_type,amount,method_pay,donation_date
0,Paul Pitts,paul.pitts@gmail.com,2012-07-16,0,None,None,0,0,1,100,1,2016-01-25
1,Claire Peters,claire.peters@gmail.com,2020-06-14,0,None,None,1,1,1,100,2,2010-02-18
2,George Beasley,george.beasley@gmail.com,2015-06-16,0,None,None,0,0,1,10,3,2016-04-27
3,Carolyn Torres,carolyn.torres@gmail.com,2012-10-17,1,Westin,CEO,1,1,1,1776,1,2016-04-27
4,Kimberly Carrillo,kimberly.carrillo@gmail.com,2020-10-08,1,Cosco,CTO,0,1,4,0,1,2016-04-27


In [32]:
result_df2[result_df2['company'] == 'Pepsi'].duplicated().sum()

27

In [27]:
result_df2.shape

(11489, 12)

In [28]:
result_df2.describe(include='object')

,name,email,date,company,role_company,donation_date
count,11489,11489,11489,11489,11489,11489
unique,992,992,484,17,4,4491
top,James Wilson,james.wilson@gmail.com,2018-11-05,None,None,2015-02-10
freq,37,37,636,3752,3752,22


In [21]:
result_df.to_csv('datos_donantes.csv', index=False)